In [7]:
import torch
import torch.nn as nn

rnn = nn.GRU(10, 20, 2, bidirectional=True)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input)#[5,3,20],
print(output.shape, hn.shape)

torch.Size([5, 3, 40]) torch.Size([4, 3, 20])


In [20]:
x = torch.randn(2,3)
y = torch.randn(2,3)
print(x,y)
output = torch.cat((x, y), dim=1)
print(output.shape)
print(output)


tensor([[ 0.4584, -1.0887,  0.5371],
        [ 1.5564, -0.6532, -0.0568]]) tensor([[-0.3872, -0.6667, -0.0190],
        [ 0.4556, -0.0068,  0.0479]])
torch.Size([2, 6])
tensor([[ 0.4584, -1.0887,  0.5371, -0.3872, -0.6667, -0.0190],
        [ 1.5564, -0.6532, -0.0568,  0.4556, -0.0068,  0.0479]])


In [55]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src): 
        '''
        src = [src_len, batch_size]
        '''
        src = src.transpose(0, 1) # src = [batch_size, src_len]
#         print("src:", src.shape)
        output = self.embedding(src)
#         print(output.shape)
        embedded = self.dropout(self.embedding(src)).transpose(0, 1) # embedded = [src_len, batch_size, emb_dim]
#         print(embedded.shape)
        # enc_output = [src_len, batch_size, hid_dim * num_directions]
        # enc_hidden = [n_layers * num_directions, batch_size, hid_dim]
        enc_output, enc_hidden = self.rnn(embedded) # if h_0 is not give, it will be set 0 acquiescently
#         print(enc_output.shape, enc_hidden.shape)#[2,3,12],[1*2,3,6]
        # enc_hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # enc_output are always from the last layer
        
        # enc_hidden [-2, :, : ] is the last of the forwards RNN 
        # enc_hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        # encoder RNNs fed through a linear layer
        # s = [batch_size, dec_hid_dim]
        output = torch.cat((enc_hidden[-2,:,:], enc_hidden[-1,:,:]), dim = 1)
#         print("output:", output.shape)#[3,12]
        s = torch.tanh(self.fc(output))#[3,7]
#         print(s.shape)
        print(enc_output.shape, s.shape)
        return enc_output, s
encoder = Encoder(7,5,6,7,0.0)
# x = torch.randn(2,3)
x = torch.tensor([[1,2,3],
    [4,5,6]])
print(x.shape)
output = encoder(x)
print("output:",output)

torch.Size([2, 3])
torch.Size([2, 3, 12]) torch.Size([3, 7])
output: (tensor([[[ 0.0098, -0.1760, -0.1943, -0.0627, -0.0699, -0.0149,  0.4709,
           0.0672, -0.2185,  0.1498, -0.2681,  0.3118],
         [-0.0568,  0.2075, -0.3382,  0.3591, -0.3153, -0.4180,  0.1708,
          -0.0518, -0.4208, -0.0412, -0.1650,  0.3432],
         [ 0.6716,  0.0550, -0.1675,  0.2383, -0.0636, -0.2470,  0.4785,
          -0.2991, -0.6967, -0.7147,  0.0309,  0.7958]],

        [[-0.0011,  0.0274, -0.5875, -0.1444, -0.2209, -0.3688,  0.3964,
           0.2409, -0.0312, -0.0054, -0.0987,  0.0791],
         [ 0.2612, -0.2060,  0.1066,  0.5450, -0.1170, -0.3733, -0.1812,
          -0.2030, -0.7021, -0.0070, -0.2136,  0.4421],
         [ 0.8378,  0.1861, -0.3021,  0.1983, -0.3633, -0.3812,  0.2773,
          -0.0250, -0.6029, -0.4994, -0.0520,  0.5525]]],
       grad_fn=<CatBackward>), tensor([[ 0.3177,  0.1279, -0.2395, -0.5031,  0.1381, -0.3423, -0.1432],
        [ 0.3131,  0.2993, -0.3893, -0.3716, -0.

In [54]:
x = torch.randn(3,2,1)
print(x)
output = x.squeeze(2)
print(output.shape)

tensor([[[ 0.3681],
         [-0.0536]],

        [[ 0.1666],
         [ 0.2726]],

        [[-1.5615],
         [-0.2276]]])
torch.Size([3, 2])
